In [76]:
import cv2
import glob
import os
from matplotlib import pyplot as plt
import numpy as num
import uuid

In [77]:
mySIFTInstance = cv2.SIFT_create()
bf = cv2.BFMatcher()

class SIFTImage:
    def __init__(self, image):
        self.image = image

        keypoints, descriptors = mySIFTInstance.detectAndCompute(image, None)
        self.keypoints = keypoints
        self.descriptors = descriptors

        self.id = uuid.uuid1()

    def __eq__(self, other):
        """Overrides the default implementation"""
        if isinstance(other, SIFTImage):
            return self.id == other.id
        return False


class MatchedImagePair:
    def __init__(self, siftImage1, siftImage2):
        self.siftImage1 = siftImage1
        self.siftImage2 = siftImage2

        self.matches = bf.knnMatch(siftImage1.descriptors, siftImage2.descriptors, k=2)

def matchedPairContains(matchedPairs, image1, image2):
    for matchedPair in matchedPairs:
        if (matchedPair.siftImage1 == image1 and matchedPair.siftImage2 == image2) or (matchedPair.siftImage1 == image2 and matchedPair.siftImage2 == image1):
            return True
    return False


In [78]:
# 1. Match Features

#The first step is to extract features from the images, and automatically establish feature correspondences between image pairs.

# The starting point is an unordered set of N images. The objective of this step is to extract features from each image, 
# and establish which (if any) of the N-1 other images contain a sufficient number of good matches so as to indicate that 
# the two images partially overlap. You can use any feature that you prefer (e.g. ORB, SIFT, SURF) or some combination thereof.
# You will need to develop a match metric that can be used to discriminate between good and bad image matches. For example, 
# this match metric could take the number of matching features between images into account, as well as the quality of their 
# match. Keep in mind that each image will only overlap with a small number of other images in the set, so ideally this match 
# metric will score high for the images that do have overlap, and low otherwise.

imagePath = os.path.expanduser('~/Autostich-Maly/D1/Images/StJames/*.jpg')  #CAN CHANGE DEPENDING ON TEST IMAGES

images = [cv2.imread(file) for file in glob.glob(imagePath)]

siftImages = []

for image in images:
    (keypoints, descriptors) = mySIFTInstance.detectAndCompute(image, None)

    siftImage = SIFTImage(image)

    siftImages.append(siftImage)

#This is test code to ensure keypoints are added correctly
# testSiftImage = siftImages[0].image
# testSiftImageKeyPoint = siftImages[0].keypoints

# imageHeight, imageWidth, imageChannels = testSiftImage.shape
# blankImage = num.zeros((imageHeight, imageWidth, imageChannels), num.uint8)
# testSiftImageOutput = cv2.drawKeypoints(testSiftImage, testSiftImageKeyPoint, blankImage)
# plt.imshow(testSiftImageOutput)

matchedPairs = []

count = 0 

for testSiftImage in siftImages:
    for siftImage in siftImages:
        if (siftImage == testSiftImage): 
            continue

        if (matchedPairContains(matchedPairs, siftImage, testSiftImage)):
            continue

        count += 1
        matchedPairs.append(MatchedImagePair(siftImage, testSiftImage))

print(count)

120


In [59]:
#this stores an array of comparisons that have aalready been performed 
#in order to prevent 'b' being tested with 'a', if 'a' had already been tested with 'b'
testedImagePairs = []



# 2. Estimate Transformation

#The second step is to use these correspondences to estimate transformations between each image, and establish the most likely transformations between the image pairs.

# For those image pairs that have a high enough match metric score from Step 1, calculate the transformation between them.



# 3. Merge Images

# The third step is to apply these transformations to compose a single merged composite image from all images.

# Apply the transformations calculated in Step 2, and create and store a single merged image from the N images in the set. 
# Apply both geometric and radiometric transformations, so that the resulting merged image appears relatively seamless.